#### train_mmaccnn_analysis
A notebook to analytically evaluate the training process for the MMAC-CNN.

In [ ]:
import os

import torch
from torch import optim
from torch.utils.data import DataLoader
from torchvision import transforms

import numpy as np
import matplotlib.pyplot as plt
from tqdm.autonotebook import tqdm

from datasets import PatchNpyDataset
from mmac_net import MMAC_CNN
from mmac_net.train_helpers import loss_acc

import train_mac as train_mmac

In [ ]:
# TODO pass required training parameters to the train_dcnn.train / train_dcnn.validate automatically
BATCH_SIZE    = 50
LR_PATIENCE   = 1  # Number of epochs of training (validation?) loss increase before the learning rate clamps down
EPOCHS        = 15 # Number of epochs to train each model
MODEL_SAMPLES = 30 # Sample size of randomly-initialized models to compare
DATA_ROOT     = '' # Root path where patch data/etc. is found.
VERBOSE       = False # Whether to print detailed output during sampling/training

A_path = 'a.npy'

In [ ]:
# Optional transforms that normalizes and augment the image patches.
train_tf = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
])

val_tf = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
])

# Load the material patch datasets
# - The testing set is only used to track its loss over time,
#   it is not used for parameter tuning / etc.
train_set     = PatchNpyDataset(root = os.path.join(DATA_ROOT, 'patch-set', 'npy', 'train'), transform = train_tf)
train_loader  = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
train_losses     = []
train_accuracies = []
print(f'Training set   : {len(train_set)} samples')

val_set     = PatchNpyDataset(root = os.path.join(DATA_ROOT, 'patch-set', 'npy', 'val'), transform = val_tf)
val_loader  = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=True)
val_losses     = []
val_accuracies = []
print(f'Validation set : {len(val_set)} samples')

test_set     = PatchNpyDataset(root = os.path.join(DATA_ROOT, 'patch-set', 'npy', 'test'), transform = val_tf)
test_loader  = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=True)
test_losses     = []
test_accuracies = []
print(f'Testing set    : {len(test_set)} samples')

In [ ]:
device_str = 'cuda' if torch.cuda.is_available() else 'cpu'
device     = torch.device(device_str)

In [ ]:
def plot_sample_losses(tr_losses, vl_losses, ts_losses, sample_idx=0):
    """Plots the training, validation, and testing losses per epoch 
    for one model sample using Matplotlib.
    
    Parameters:
        tr_losses: np.array
            Total training loss per epoch.
        vl_losses: np.array
            Total validation loss per epoch.
        ts_losses: np.array
            Total testing losses per epoch.
        sample_idx: int (optional)
            The index of the desired sample. The default is 0.
    """
    fig, ax = plt.subplots()
    plt.title('MAC-CNN losses per epoch')
    
    x = np.arange(0, EPOCHS)
    ax.plot(x, tr_losses[sample_idx,:], '.-', label='Training loss', color='blue')
    ax.plot(x, vl_losses[sample_idx,:], '.-', label='Validation loss', color='orange')
    ax.plot(x, ts_losses[sample_idx,:], '.-', label='Testing loss', color='green')
    ax.set_xlim(0)
    ax.set_ylim(0)

    ax.set_xlabel('Epoch')
    ax.set_ylabel('Total loss')
    ax.legend()
    
    plt.savefig('maccnn-epoch-losses.eps', dpi=192, format='eps')
    #plt.savefig('maccnn-epoch-losses.png', dpi=300, format='png')
    plt.show()

In [ ]:
def plot_sample_accuracies(tr_accuracies, vl_accuracies, ts_accuracies, sample_idx=0):
    """Plots the training, validation, and testing accuracies per epoch 
    for one model sample using Matplotlib.
    
    Parameters:
        tr_accuracies: np.array
            Total training acccuracies per epoch.
        vl_accuracies: np.array
            Total validation accuracies per epoch.
        ts_accuracies: np.array
            Total testing accuracies per epoch.
        sample_idx: int (optional)
            The index of the desired sample. The default is 0.
    """
    fig, ax = plt.subplots()
    plt.title('MAC-CNN accuracies per epoch')
    
    x = np.arange(0, EPOCHS)
    ax.plot(x, tr_accuracies[sample_idx,:], '.-', label='Training accuracy', color='blue')
    ax.plot(x, vl_accuracies[sample_idx,:], '.-', label='Validation accuracy', color='orange')
    ax.plot(x, ts_accuracies[sample_idx,:], '.-', label='Testing accuracy', color='green')
    
    ax.set_xlim(xmin=0)
    ax.set_ylim(ymin=0, ymax=100)
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Accuracy (%)')
    ax.legend()
    
    plt.savefig('maccnn-epoch-accuracies.eps', dpi=192, format='eps')
    #plt.savefig('mmaccnn-epoch-accuracies.png', dpi=300, format='png')
    plt.show()

In [ ]:
def plot_distribution_losses(tr_losses, vl_losses, ts_losses):
    """Plots the distributions of training, validation, and testing
    losses per epoch for all model samples using Matplotlib.
    
    Parameters:
        tr_losses: np.array
            Total training loss per epoch.
        vl_losses: np.array
            Total validation loss per epoch.
        ts_losses: np.array
            Total testing losses per epoch.
    """
    fig, ax = plt.subplots()
    plt.title('MAC-CNN losses per epoch')
    
    # Build median lines
    med_tr_losses = np.median(tr_losses, axis=0)
    med_vl_losses = np.median(vl_losses, axis=0)
    med_ts_losses = np.median(ts_losses, axis=0)
    
    # Build 75th percentile lines
    p75_tr_losses = np.percentile(tr_losses, 75, axis=0)
    p75_vl_losses = np.percentile(vl_losses, 75, axis=0)
    #p75_ts_losses = np.percentile(ts_losses, 75, axis=0)
    
    # Build 25th percentile lines
    p25_tr_losses = np.percentile(tr_losses, 25, axis=0)
    p25_vl_losses = np.percentile(vl_losses, 25, axis=0)
    #p25_ts_losses = np.percentile(ts_losses, 25, axis=0)
    
    x = np.arange(0, EPOCHS)

    # Plot median lines
    ax.plot(x, med_tr_losses, '.-', label='Training loss', color='blue')
    ax.plot(x, med_vl_losses, '.-', label='Validation loss', color='orange')
    #ax.plot(x, med_ts_losses, '.-', label='Testing loss', color='green')
    
    # Plot 25th-75th percentile shaded regions
    ax.fill_between(x, p25_tr_losses, p75_tr_losses, facecolor='blue', alpha=0.2)
    ax.fill_between(x, p25_vl_losses, p75_vl_losses, facecolor='orange',  alpha=0.2)
    #ax.fill_between(x, p25_ts_losses, p75_ts_losses, facecolor='green', alpha=0.2)
    
    ax.set_xlim(0)
    ax.set_ylim(0)
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Total loss')
    ax.legend()
    
    #plt.savefig('mmaccnn-epoch-losses-dist.eps', dpi=192, format='eps')
    plt.savefig('maccnn-epoch-losses-dist.png', dpi=192, format='png')
    plt.show()

In [ ]:
def plot_distribution_accuracies(tr_accuracies, vl_accuracies, ts_accuracies):
    """Plots the distributions of training, validation, and testing 
    accuracies per epoch for all model samples using Matplotlib.
    
    Parameters:
        tr_accuracies: np.array
            Total training acccuracies per epoch.
        vl_accuracies: np.array
            Total validation accuracies per epoch.
        ts_accuracies: np.array
            Total testing accuracies per epoch.
    """
    fig, ax = plt.subplots()
    plt.title('MAC-CNN accuracies per epoch')
       
    # Build median lines
    med_tr_accuracies = np.median(tr_accuracies, axis=0)
    med_vl_accuracies = np.median(vl_accuracies, axis=0)
    med_ts_accuracies = np.median(ts_accuracies, axis=0)
    
    # Build 75th %ile lines
    p75_tr_accuracies = np.percentile(tr_accuracies, 75, axis=0)
    p75_vl_accuracies = np.percentile(vl_accuracies, 75, axis=0)
    p75_ts_accuracies = np.percentile(ts_accuracies, 75, axis=0)
    
    # Build 25th %ile lines
    p25_tr_accuracies = np.percentile(tr_accuracies, 25, axis=0)
    p25_vl_accuracies = np.percentile(vl_accuracies, 25, axis=0)
    p25_ts_accuracies = np.percentile(ts_accuracies, 25, axis=0)
    
    x = np.arange(0, EPOCHS)

    # Plot median lines
    ax.plot(x, med_tr_accuracies, '.-', label='Training accuracy', color='blue')
    ax.plot(x, med_vl_accuracies, '.-', label='Validation accuracy', color='orange')
    #ax.plot(x, med_ts_accuracies, '.-', label='Testing loss', color='green')
    
    # Plot 25th-75th percentile shaded regions
    ax.fill_between(x, p25_tr_accuracies, p75_tr_accuracies, facecolor='blue', alpha=0.2)
    ax.fill_between(x, p25_vl_accuracies, p75_vl_accuracies, facecolor='orange', alpha=0.2)
    #ax.fill_between(x, p25_ts_accuracies, p75_ts_accuracies, facecolor='green', alpha=0.2)
    
    ax.set_xlim(xmin=0)
    ax.set_ylim(ymin=0, ymax=100)
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Accuracy (%)')
    ax.legend()
    
    #plt.savefig('mmaccnn-epoch-accuracies-dist.eps', dpi=192, format='eps')
    plt.savefig('maccnn-epoch-accuracies-dist.png', dpi=192, format='png')
    plt.show()

In [ ]:
def test(model, device, loader, w_per = 1e0, w_kld = 1e-2, verbose=True):
    """Tests the MMAC-CNN.
    
    Parameters:
        model: torch.nn
            The neural network being tested.
        device: string
            The device (cpu or cuda) that the model is being run on.
        test_loader: Dataloader
            The test data.
        verbose: bool (optional)
            If True, prints out information (in addition to progress 
            bars) from the function.
    """
    model.eval()
    
    A = model.A
    total_loss  = 0.0
    total_acc   = 0.0
    total_ucost = 0.0
    
    for batch_idx, batch in enumerate(tqdm(test_loader, unit=' testing batches')):
        X, y = batch

        X = X.to(device)
        y = y.to(device)
        
        # Forward pass
        # y_pred is the k class prediction
        # A_preds are the a1, a2, ..., a5, a_final m attribute predictions
        # from each level of the ResNet auxillary layers
        y_pred, A_preds = model(X)   
        loss, acc, u_cost = loss_acc(y, y_pred, A, A_preds, w_kld, w_per)
        
        total_loss  += float(loss)
        total_acc   += float(acc)
        total_ucost += float(u_cost)

    # Print results
    if verbose:
        print(f'Test loss  : {test_loss / len(loader)}')
        print(f'Test ucost : {test_ucost / len(loader)}')
        print(f'Test acc   : {test_acc / len(loader) * 100:.3f}%')
    
    return total_loss, total_acc / len(loader), total_ucost / len(loader)

In [ ]:
print(f'Labels   : {train_set.get_labels()}')
print(f'Device   : {device_str}')
print(f'Verbose? : {VERBOSE}')

train_losses = np.zeros((MODEL_SAMPLES, EPOCHS))
val_losses   = np.zeros((MODEL_SAMPLES, EPOCHS))
test_losses  = np.zeros((MODEL_SAMPLES, EPOCHS))

train_accuracies = np.zeros((MODEL_SAMPLES, EPOCHS))
val_accuracies   = np.zeros((MODEL_SAMPLES, EPOCHS))
test_accuracies  = np.zeros((MODEL_SAMPLES, EPOCHS))

train_ucosts = np.zeros((MODEL_SAMPLES, EPOCHS))
val_ucosts   = np.zeros((MODEL_SAMPLES, EPOCHS))
test_ucosts  = np.zeros((MODEL_SAMPLES, EPOCHS))

min_loss = float('inf')    # Lowest loss of the D-CNN on the validation set of all epochs (and model samples) so far
A        = np.load(A_path) # A matrix for the constrained linear layers

for s in range(MODEL_SAMPLES):
    print(f'\n---------------- SAMPLE {s+1}/{MODEL_SAMPLES} ---------------- ')
    
    mmac_cnn  = MMAC_CNN(A, 32).to(device) # Create a new model sample (Second parameter assumes the image patches are 32x32 px)
    optimizer = optim.Adam(mmac_cnn.parameters(), lr=1e-3)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=LR_PATIENCE, min_lr=1e-8)
    
    for e in range(EPOCHS):
        print(f'~~~~ EPOCH {e+1}/{EPOCHS} ~~~~')
        print(f'lr: {optimizer.param_groups[0]["lr"]}')

        train_loss, acc, ucost = train_mmac.train(
            mmac_cnn, device, train_loader, optimizer, 
            verbose=VERBOSE
        )
        
        train_losses[s, e]     = train_loss
        train_accuracies[s, e] = acc * 100.0
        train_ucosts[s, e]     = ucost

        val_loss, acc, ucost = train_mmac.validate(
            mmac_cnn, device, val_loader, verbose=VERBOSE
        )
        
        val_losses[s, e]     = val_loss
        val_accuracies[s, e] = acc * 100.0
        val_ucosts[s, e]     = ucost

        if s == 0:
            # Only run the testing set of the first sample, to show how extremely close
            # testing and validation loss are.
            test_loss, acc, ucost = test(
                mmac_cnn, device, test_loader, verbose=VERBOSE
            )
            
            test_losses[s, e]     = test_loss
            test_accuracies[s, e] = acc * 100.0
            test_ucosts[s, e]     = ucost

        # If this is the lowest-loss (and therefore generally most accurate)
        # run so far out of ALL model samples, save the MMAC-CNN model weights to disk
        if (val_loss < min_loss):
            min_loss = val_loss
            print('New minimum validation loss:', min_loss)
            print('Saving mmac_cnn state to mmac_cnn.pt...')
            torch.save(mmac_cnn, 'mmac_cnn.pt')      
        scheduler.step(val_loss)
    
    # Free CUDA memory
    del mmac_cnn
    del optimizer
    del scheduler

In [ ]:
# After done, plot the training and testing losses for the first sample (idx 0) on MatPlotLib
plot_sample_losses(train_losses, val_losses, test_losses, sample_idx=0)
plot_sample_accuracies(train_accuracies, val_accuracies, test_accuracies, sample_idx=0)

In [ ]:
plot_distribution_losses(train_losses, val_losses, test_losses)
plot_distribution_accuracies(train_accuracies, val_accuracies, test_accuracies)

print('Train losses:\n', train_losses)
print('\nVal losses:\n', val_losses)

print('Train accuracies:\n', train_accuracies)
print('\nVal accuracies:\n', val_accuracies)

In [ ]:
np.save('mmaccnn_train_losses.npy', train_losses)
np.save('mmaccnn_val_losses.npy', val_losses)
np.save('mmaccnn_test_losses.npy', test_losses)

np.save('mmaccnn_train_accuracies.npy', train_accuracies)
np.save('mmaccnn_val_accuracies.npy', val_accuracies)
np.save('mmaccnn_test_accuracies.npy', test_accuracies)